In [1]:
import pandas as pd
df=pd.read_csv(r"C:\Users\tareq\Desktop\p2\Data\cleaned.csv")
label=pd.read_excel(r"C:\Users\tareq\Desktop\p2\Data\Labeled.csv")

In [3]:
df2_cols = df.columns.tolist()
df2_cols.remove('Username')
df2 = df[['Username'] + df2_cols]

# Merge the two dataframes on the 'username' column
result = pd.merge(label, df2, on='Username', how='left')
all=[]
for i in range(len(result)): 
    label=[]
    label.append(int(result['Label'][i].split()[0][1]))
    label.append(int(result['Label'][i].split()[1]))
    label.append(int(result['Label'][i].split()[2]))
    label.append(int(result['Label'][i].split()[3]))
    label.append(int(result['Label'][i].split()[4][0]))
    all.append(label)
result['LABEL']=all
result.drop('Label',axis=1,inplace=True)
df=result.copy()
df['OpennesS']=[x[0] for x in df['LABEL']]
df['ConscientiousnesS']=[x[1] for x in df['LABEL']]
df['ExtraversioN']=[x[2] for x in df['LABEL']]
df['AgreeablenesS']=[x[3] for x in df['LABEL']]
df['NeuroticisM']=[x[4] for x in df['LABEL']]
df.drop(['Username','LABEL'],axis=1,inplace=True)

In [14]:
col= df.columns.tolist()
col.remove('ExtraversioN')
col.remove('OpennesS')
col.remove('ConscientiousnesS')
col.remove('AgreeablenesS')
col.remove('NeuroticisM')
dfO = df[['OpennesS'] + col]
dfE=df[['ExtraversioN'] + col]
dfC=df[['ConscientiousnesS'] + col]
dfN=df[['NeuroticisM'] + col]
dfA=df[['AgreeablenesS'] + col]

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(dfO['Tweet'])

In [20]:
seq=tokenizer.texts_to_sequences(dfO['Tweet'])

In [26]:
len(seq[100])

311

In [27]:
idx_word=tokenizer.index_word

In [30]:
max_seq_len=max([len(x) for x in seq])

In [31]:
max_seq_len

5227

In [33]:
import numpy as np

In [34]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
seqP=  np.array(pad_sequences(seq,maxlen=max_seq_len,padding='pre'))

In [38]:
labels = df['OpennesS']

In [40]:
num_words=len(idx_word)+1

In [70]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(seqP, labels, test_size=0.15,random_state=42)

In [64]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

In [71]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(num_words, embedding_vecor_length, input_length=max_seq_len))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=64)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 5227, 32)          4798592   
                                                                 
 lstm_6 (LSTM)               (None, 100)               53200     
                                                                 
 dense_10 (Dense)            (None, 1)                 101       
                                                                 
Total params: 4,851,893
Trainable params: 4,851,893
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
7/7 [==============================] - 535s 78s/step - loss: 0.6838 - accuracy: 0.6629 - val_loss: 0.6579 - val_accuracy: 0.7375
Epoch 2/3
7/7 [==============================] - 587s 85s/step - loss: 0.6160 - accuracy: 0.7031 - val_loss: 0.5722 - val_accuracy: 0.7375
Epoch 3/3
7/7 [==========

In [73]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
model1 = Sequential()
model1.add(Embedding(num_words, embedding_vecor_length, input_length=max_seq_len))
model1.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model1.add(MaxPooling1D(pool_size=2))
model1.add(LSTM(100))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())
model1.fit(x_train, y_train,validation_data=(x_test, y_test), epochs=3, batch_size=64)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 5227, 32)          4798592   
                                                                 
 conv1d (Conv1D)             (None, 5227, 32)          3104      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2613, 32)         0         
 )                                                               
                                                                 
 lstm_7 (LSTM)               (None, 100)               53200     
                                                                 
 dense_11 (Dense)            (None, 1)                 101       
                                                                 
Total params: 4,854,997
Trainable params: 4,854,997
Non-trainable params: 0
____________________________________________

KeyboardInterrupt: 

In [78]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
m=RandomForestClassifier(max_depth= 15, max_features='sqrt', n_estimators= 1000)
m.fit(x_train,y_train)
y_pred=m.predict(x_test)
accuracy_score(y_test,y_pred)

0.75

In [445]:
import pickle
filename = 'O75.sav'
pickle.dump(m, open(filename, 'wb'))

In [402]:
labels = df['ConscientiousnesS']
x_train, x_test, y_train, y_test = train_test_split(seqP, labels, test_size=0.15,random_state=42)

In [222]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error


param_grid = {'n_estimators': [100,500, 800],
              'max_depth': [5, 8, 10],
              'max_features': ['sqrt', 'log2']}

rfr = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rfr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 8, 40],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [500, 800, 1000]},
             scoring='neg_mean_squared_error')

In [223]:
best_params = grid_search.best_params_
best_mse = abs(grid_search.best_score_)
print('Best parameters:', best_params)
print('Best MSE:', best_mse)

Best parameters: {'max_depth': 5, 'max_features': 'log2', 'n_estimators': 1000}
Best MSE: 0.23285351757257317


In [430]:
m1=RandomForestClassifier(max_depth= 5, max_features='log2', n_estimators= 1000)
m1.fit(x_train,y_train)
y_pred=m1.predict(x_test)

In [431]:
accuracy_score(y_test,y_pred)

0.7125

In [444]:
import pickle
filename = 'C71.sav'
pickle.dump(m1, open(filename, 'wb'))

In [414]:
labels = df['ExtraversioN']

In [390]:
dfE=pd.DataFrame(seqP)
dfE['Label']=labels
dfE1=dfE[dfE['Label']==1]
dfE0=dfE[dfE['Label']==0]
dfE0=dfE0.sample(n=113)
dfEB=pd.concat([dfE0,dfE1])
dfEB = dfEB.sample(frac=1).reset_index(drop=True)
labels=dfEB['Label']
seq=dfEB.drop('Label',axis=1)
x_train, x_test, y_train, y_test = train_test_split(seq, labels, test_size=0.15,random_state=42)

In [392]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error


param_grid = {'n_estimators': [50,100,150],
              'max_depth': [1,3,5,7,9],
              'max_features': ['sqrt', 'log2']}

rfr = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rfr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [1, 3, 5, 7, 9],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [50, 100, 150]},
             scoring='neg_mean_squared_error')

In [393]:
best_params = grid_search.best_params_
best_mse = abs(grid_search.best_score_)
print('Best parameters:', best_params)
print('Best MSE:', best_mse)

Best parameters: {'max_depth': 7, 'max_features': 'log2', 'n_estimators': 50}
Best MSE: 0.25256127586148047


In [ ]:
m7=RandomForestClassifier(max_depth= 7, max_features='log2', n_estimators= 50)
m7.fit(x_train,y_train)
y_pred=m7.predict(x_test)

In [401]:
accuracy_score(y_test,y_pred)

0.7941176470588235

In [443]:
import pickle
filename = 'E79.sav'
pickle.dump(m7, open(filename, 'wb'))

In [432]:
labels = df['AgreeablenesS']

In [433]:
dfA=pd.DataFrame(seqP)
dfA['Label']=labels
dfA1=dfA[dfA['Label']==1]
dfA0=dfA[dfA['Label']==0]
dfA0=dfA0.sample(n=151)
dfAB=pd.concat([dfA0,dfA1])
dfAB = dfAB.sample(frac=1).reset_index(drop=True)
labels=dfAB['Label']
seq=dfAB.drop('Label',axis=1)
x_train, x_test, y_train, y_test = train_test_split(seq, labels, test_size=0.15,random_state=42)

In [371]:
param_grid = {'n_estimators': [100,500,1000],
              'max_depth': [5,10,15],
              'max_features': ['sqrt', 'log2']}

rfr = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rfr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [100, 500, 1000]},
             scoring='neg_mean_squared_error')

In [372]:
best_params = grid_search.best_params_
best_mse = abs(grid_search.best_score_)
print('Best parameters:', best_params)
print('Best MSE:', best_mse)

Best parameters: {'max_depth': 5, 'max_features': 'log2', 'n_estimators': 1000}
Best MSE: 0.260572212699426


In [434]:
m4=RandomForestClassifier(max_depth= 5, max_features='log2', n_estimators= 200)
m4.fit(x_train,y_train)
y_pred=m4.predict(x_test)
accuracy_score(y_test,y_pred)
#0.73

0.3695652173913043

In [440]:
accuracy_score(y_test,y_pred)

0.7391304347826086

In [442]:
import pickle
filename = 'A74.sav'
pickle.dump(m4, open(filename, 'wb'))